In [2]:
import re
from collections import defaultdict as dd, Counter
from nltk.util import bigrams
import nltk
from sklearn.model_selection import train_test_split as tts
import random
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:

with open('transcript.txt', 'r') as f:
  transcript = [re.split(r'(\d+\w\d\w\d+)', l)[-1].strip() for l in f.readlines()]

In [4]:

# divide data 80 : 20
train_transcript, test_transcipt =  tts(transcript, test_size=0.2, random_state=42)

In [5]:
# replace low frequency word with <UNK> in train set
word_counter = Counter()
for sentence in train_transcript:
  tokens = nltk.word_tokenize(sentence)
  word_counter.update(tokens)

threshold = 10

In [6]:

def replace_word_with_unk(tokens, threshold, word_counter):
  return [token if word_counter[token] > threshold else '<unk>' for token in tokens]

sentences_with_unk = []
for sentence in train_transcript:
  tokens = nltk.word_tokenize(sentence)
  sentences_with_unk.append(replace_word_with_unk(tokens, threshold, word_counter))

In [7]:

# build bigram, unigram table
bigram_model = dd(lambda: dd(lambda: 0))
unigram_model = dd(lambda: 0)
total_token = 0

for sentence in sentences_with_unk:
  # bigram
  for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
    bigram_model[w1][w2] += 1
  # unigram
  for w in sentence:
    unigram_model[w] += 1
    total_token += 1

# normalize
for w in unigram_model.keys():
  unigram_model[w] /= total_token 

for w1 in bigram_model:
  total_count = sum(bigram_model[w1].values())
  for w2 in bigram_model[w1]: 
    bigram_model[w1][w2] /= total_count


In [9]:
def generate_sentences(bigram_model_) -> str:
  result = ''
  word = None
  while True:
    # get next word base on current word
    word = random.choices(list(bigram_model_[word].keys()), weights=list(bigram_model_[word].values()), k=1)[0]
    result += ' ' + word if word != None else ''
    # end sentences with None token
    if word == None: break
  return result

# generate 5 sentences
for i in range(5):
  print(''.join(generate_sentences(bigram_model)))

 there is good place
 how about chez-panisse * * ethiopian * > <unk> and i wan na make a restaurant on sunday
 how about ten dollars
 okay i 'd like to eat . <unk> chinese food
 i 'd like some more than ten dollars
